In [1]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, KFold, RandomizedSearchCV, train_test_split
from sklearn import datasets, ensemble
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pandas as pd
import numpy as np

from tqdm import tqdm
from scipy.signal import find_peaks
from scipy.stats import mannwhitneyu
from scipy import stats
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score, precision_score,confusion_matrix

import pandas as pd
from scipy.stats import ttest_ind

In [2]:
from decimal import Decimal, ROUND_HALF_UP

def round_v3(num, decimal):
    str_deci = 1
    for _ in range(decimal):
        str_deci = str_deci / 10
    str_deci = str(str_deci)
    result = Decimal(str(num)).quantize(Decimal(str_deci), rounding=ROUND_HALF_UP)
    result = float(result)
    print(result)
    return result

In [37]:
file_path = 'C:/Users/Jackson/OneDrive/LAB/資料/excel/Warcar/第二次/壓力General_Warcar_Stroop.xlsx'
# file_path = 'C:/Users/Jackson/OneDrive/LAB/資料/excel/Warcar/第二次/壓力General_Warcar_CPT.xlsx'
# file_path = 'C:/Users/Jackson/OneDrive/LAB/資料/excel/Warcar/第一次/壓力General_Warcar_Stroop.xlsx'
# file_path = 'C:/Users/Jackson/OneDrive/LAB/資料/excel/Warcar/第一次/壓力General_Warcar_CPT.xlsx'
# file_path = 'C:/Users/Jackson/OneDrive/LAB/資料/excel/Warcar/壓力General_Warcar_CPT.xlsx'
# file_path = 'C:/Users/Jackson/OneDrive/LAB/資料/excel/Warcar/壓力General_Warcar_Stroop_3min.xlsx'
# file_path = 'C:/Users/Jackson/OneDrive/LAB/資料/excel/Warcar/壓力General_Warcar_CPT_3min.xlsx'
data = pd.read_excel(file_path)
x = []
y = []
x = data[['PSS', 'blink_duration_m', 'blink_duration_std', 'blink_interval_m', 'blink_interval_std', 'diam_mean', 'diam_std', 'diam_delta_mean', 'diam_delta_std', 'pcps_mean',
          'blink_rate', 'fixation_count', 'gaze_angle', 'path', 'mrri', 'sdnn','sdsd', 'nn50','pnn50', 'nn20', 'pnn20', 'rmssd', 'mhr','sdhr',
           'ulf_p', 'vlf_p','lf_p', 'hf_p', 'lf_hf', 'lfnu', 'hfnu', 'total_p', 'cvi', 
           'sd1', 'sd2', 'sampen', 'heart_mean', 'heart_std', 'heart_var', 'heart_growth', 'stresslv_mean', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
           , 'spo2_mean', 'spo2_std', 'spo2_var', 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth'
            , 'answer_freq', 'correct', 'wrong', 'score', 'collision_count',
            'FC3_delta', 'FC3_theta', 'FC3_alpha', 'FC3_beta', 'FC3_gamma', 'FCz_delta', 'FCz_theta', 'FCz_alpha', 'FCz_beta', 'FCz_gamma', 'Pz_delta',
            'Pz_theta', 'Pz_alpha', 'Pz_beta', 'Pz_gamma', 'Oz_delta', 'Oz_theta', 'Oz_alpha', 'Oz_beta', 'Oz_beta']].values.tolist()
y = data[['phase']].values.tolist()
x = np.array(x)
y = np.array(y)
print(x.shape)
print(y.shape)

(60, 77)
(60, 1)


c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)


In [38]:
feature_names = ['PSS', 'blink_duration_m', 'blink_duration_std', 'blink_interval_m', 'blink_interval_std', 'diam_mean', 'diam_std', 'diam_delta_mean', 'diam_delta_std', 'pcps_mean',
          'blink_rate', 'fixation_count', 'gaze_angle', 'path', 'mrri', 'sdnn','sdsd', 'nn50','pnn50', 'nn20', 'pnn20', 'rmssd', 'mhr','sdhr',
           'ulf_p', 'vlf_p','lf_p', 'hf_p', 'lf_hf', 'lfnu', 'hfnu', 'total_p', 'cvi', 
           'sd1', 'sd2', 'sampen', 'heart_mean', 'heart_std', 'heart_var', 'heart_growth', 'stresslv_mean', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
           , 'spo2_mean', 'spo2_std', 'spo2_var', 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth'
            , 'answer_freq', 'correct', 'wrong', 'score', 'collision_count',
            'FC3_delta', 'FC3_theta', 'FC3_alpha', 'FC3_beta', 'FC3_gamma', 'FCz_delta', 'FCz_theta', 'FCz_alpha', 'FCz_beta', 'FCz_gamma', 'Pz_delta',
            'Pz_theta', 'Pz_alpha', 'Pz_beta', 'Pz_gamma', 'Oz_delta', 'Oz_theta', 'Oz_alpha', 'Oz_beta', 'Oz_beta']

In [39]:
# 將 y 轉換為一維數組
y = y.ravel()
# 將 x 按照 y 的值（phase）分成兩個組
x_phase1 = x[y == 1]
# x_phase1 = x[(y == 1) | (y == 3)]
x_phase2 = x[y == 2]
# 初始化存放 t 檢定結果的列表
t_test_results = []

In [40]:
#             , 'correct', 'wrong', 'score']].values.tolist()
y = data[['phase']].values.tolist()
x = np.array(x)
y = np.array(y)
# 將 phase=1 和 phase=2 的資料標記為 label 0 和 label 1
y_binary = np.where(np.array(y) == 1, 0, np.where(np.array(y) == 2, 1, -1))  # phase=3標記為-1

# 過濾掉 phase=3 的資料
valid_indices = np.where(y_binary != -1)[0]
x_filtered = x[valid_indices]
y_filtered = y_binary[valid_indices]
print(x_filtered.shape)
print(y_filtered)

(40, 77)
[[0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]]


In [ ]:
# # 初始化存放 U 檢定結果的列表
# u_test_results = []

# # 進行 U 檢定
# for i in range(x.shape[1]):
#     u_statistic, p_value = mannwhitneyu(x_phase1[:, i], x_phase2[:, i], alternative='two-sided')
#     if p_value < 0.05:  # 如果 p-value 小於 0.05，表示差異顯著
#         mean_x_phase1 = np.mean(x_phase1[:, i])
#         mean_x_phase2 = np.mean(x_phase2[:, i])
#         u_test_results.append((feature_names[i], u_statistic, p_value, mean_x_phase1, mean_x_phase2))

# # 將結果轉換為 DataFrame
# result_df = pd.DataFrame(u_test_results, columns=['Feature', 'U-Statistic', 'P-Value', 'mean_x_phase1', 'mean_x_phase2'])

# # 列印有顯著差異的特徵
# print(result_df)

In [ ]:
# # 初始化存放 t 檢定結果的列表
# t_results = []
# # 進行 t 檢定
# for i in range(x.shape[1]):
#     t_statistic, p_value = ttest_ind(x_phase1[:, i], x_phase2[:, i])
#     if p_value < 0.05:  # 如果 p-value 小於 0.05，表示差異顯著
#         mean_x_phase1 = np.mean(x_phase1[:, i])
#         mean_x_phase2 = np.mean(x_phase2[:, i])
#         t_results.append((feature_names[i], t_statistic, p_value, mean_x_phase1, mean_x_phase2))

# # 將結果轉換為 DataFrame
# result_df = pd.DataFrame(t_results, columns=['Feature', 'T-Statistic', 'P-Value', 'mean_x_phase1', 'mean_x_phase2'])

# # 列印有顯著差異的特徵
# print(result_df)

In [41]:
from scipy.stats import ttest_rel
# 初始化存放 t 檢定結果的列表
t_results = []
# 進行 t 檢定
for i in range(x.shape[1]):
    t_statistic, p_value = ttest_rel(x_phase1[:, i], x_phase2[:, i])
    # if p_value < 0.05:  # 如果 p-value 小於 0.05，表示差異顯著
    mean_x_phase1 = np.mean(x_phase1[:, i])
    mean_x_phase2 = np.mean(x_phase2[:, i])
    t_results.append((feature_names[i], t_statistic, p_value, mean_x_phase1, mean_x_phase2))

# 將結果轉換為 DataFrame
result_df = pd.DataFrame(t_results, columns=['Feature', 'T-Statistic', 'P-Value', 'mean_x_phase1', 'mean_x_phase2'])

# 列印有顯著差異的特徵
print(result_df)

               Feature  T-Statistic   P-Value  mean_x_phase1  mean_x_phase2
0                  PSS    -6.543190  0.000003       8.900000      12.150000
1     blink_duration_m    -0.059209  0.953404     175.027056     175.442390
2   blink_duration_std     0.140234  0.889951     127.123789     124.102735
3     blink_interval_m    -0.512850  0.613967    8697.940998   11468.942278
4   blink_interval_std    -0.645942  0.526042    8287.832656   10863.141178
..                 ...          ...       ...            ...            ...
72            Oz_delta    -0.073046  0.942533       0.655027       0.655924
73            Oz_theta     0.320332  0.752213       0.192144       0.189929
74            Oz_alpha    -0.448094  0.659153       0.065973       0.067331
75             Oz_beta    -0.617027  0.544541       0.072594       0.074279
76             Oz_beta    -0.617027  0.544541       0.072594       0.074279

[77 rows x 5 columns]


In [42]:
result_df.to_csv('C:/Users/Jackson/Downloads/result.csv', index=False)
result_df

,Feature,T-Statistic,P-Value,mean_x_phase1,mean_x_phase2
0,PSS,-6.543190,0.000003,8.900000,12.150000
1,blink_duration_m,-0.059209,0.953404,175.027056,175.442390
2,blink_duration_std,0.140234,0.889951,127.123789,124.102735
3,blink_interval_m,-0.512850,0.613967,8697.940998,11468.942278
4,blink_interval_std,-0.645942,0.526042,8287.832656,10863.141178
...,...,...,...,...,...
72,Oz_delta,-0.073046,0.942533,0.655027,0.655924
73,Oz_theta,0.320332,0.752213,0.192144,0.189929
74,Oz_alpha,-0.448094,0.659153,0.065973,0.067331
75,Oz_beta,-0.617027,0.544541,0.072594,0.074279


In [24]:
mean = []
for feature_index in range(x.shape[1]):
    feature_phase1 = x_phase1[:, feature_index]
    feature_phase2 = x_phase2[:, feature_index]
    mean_x_phase1 = np.mean(feature_phase1)
    mean_x_phase2 = np.mean(feature_phase2)
    mean.append((feature_names[feature_index], mean_x_phase1, mean_x_phase2))

# 打印結果並保存到 CSV 文件
result_df = pd.DataFrame(mean, columns=['Feature', 'LowPressure', 'HighPressure'])

In [25]:
result_df.to_csv('C:/Users/Jackson/Downloads/result.csv', index=False)
result_df

,Feature,LowPressure,HighPressure
0,PSS,10.050000,15.300000
1,blink_duration_m,175.027056,175.442390
2,blink_duration_std,127.123789,124.102735
3,blink_interval_m,8697.940998,11468.942278
4,blink_interval_std,8287.832656,10863.141178
...,...,...,...
72,Oz_delta,0.655027,0.655924
73,Oz_theta,0.192144,0.189929
74,Oz_alpha,0.065973,0.067331
75,Oz_beta,0.072594,0.074279


In [145]:
# x = data[['PSS', 'blink_duration_m', 'blink_duration_std', 'blink_interval_m', 'blink_interval_std', 'diam_mean', 'diam_std', 'diam_delta_mean', 'diam_delta_std', 'pcps_mean',
#           'blink_rate', 'fixation_count', 'gaze_angle', 'path', 'mrri', 'sdnn','sdsd', 'nn50','pnn50', 'nn20', 'pnn20', 'rmssd', 'mhr','sdhr',
#            'ulf_p', 'vlf_p','lf_p', 'hf_p', 'lf_hf', 'lfnu', 'hfnu', 'total_p', 'cvi', 
#            'sd1', 'sd2', 'sampen', 'heart_mean', 'heart_std', 'heart_var', 'heart_growth', 'stresslv_mean', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
#            , 'spo2_mean', 'spo2_std', 'spo2_var', 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth'
#             , 'answer_freq', 'correct', 'wrong', 'score', 'collision_count',
#             'FC3_delta', 'FC3_theta', 'FC3_alpha', 'FC3_beta', 'FC3_gamma', 'FCz_delta', 'FCz_theta', 'FCz_alpha', 'FCz_beta', 'FCz_gamma', 'Pz_delta',
#             'Pz_theta', 'Pz_alpha', 'Pz_beta', 'Pz_gamma', 'Oz_delta', 'Oz_theta', 'Oz_alpha', 'Oz_beta', 'Oz_beta']].values.tolist()

# stroop顯著
# x = data[['PSS','heart_growth','stresslv_std', 'stresslv_var', 'stresslv_growth'
#            , 'collision_count']].values.tolist()
# x = data[['PSS', 'sdhr', 'heart_growth', 'stresslv_std', 'stresslv_var', 'answer_freq', 'correct', 'wrong', 'score', 'collision_count',
#             'FC3_gamma', 'FCz_gamma', 'Pz_gamma']].values.tolist()

# CTP顯著
# x = data[['PSS','nn50', 'nn20', 'heart_std', 'heart_var', 'heart_growth', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
#            ,  'resp_std', 'resp_var', 'score']].values.tolist()
# x = data[['PSS', 'diam_mean', 'blink_rate', 'sdnn', 'nn50','pnn50', 'nn20', 'pnn20', 'heart_std', 'heart_growth', 'stresslv_mean', 'stresslv_std', 'stresslv_var'
#            , 'resp_std', 'resp_var', 'correct', 'wrong', 'score']].values.tolist()


# x = data[['FC3_delta', 'FC3_theta', 'FC3_alpha', 'FC3_beta', 'FC3_gamma', 'FCz_delta', 'FCz_theta', 'FCz_alpha', 'FCz_beta', 'FCz_gamma', 'Pz_delta',
#             'Pz_theta', 'Pz_alpha', 'Pz_beta', 'Pz_gamma', 'Oz_delta', 'Oz_theta', 'Oz_alpha', 'Oz_beta', 'Oz_beta']].values.tolist()

# x = data[['blink_duration_m', 'blink_duration_std', 'blink_interval_m', 'blink_interval_std', 'diam_mean', 'diam_std', 'diam_delta_mean', 'diam_delta_std', 'pcps_mean',
#           'blink_rate', 'fixation_count', 'gaze_angle', 'path']].values.tolist()

# x = data[['mrri', 'sdnn','sdsd', 'nn50','pnn50', 'nn20', 'pnn20', 'rmssd', 'mhr','sdhr',
#            'ulf_p', 'vlf_p','lf_p', 'hf_p', 'lf_hf', 'lfnu', 'hfnu', 'total_p', 'cvi', 
#            'sd1', 'sd2', 'sampen', 'heart_mean', 'heart_std', 'heart_var', 'heart_growth', 'stresslv_mean', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
#            , 'spo2_mean', 'spo2_std', 'spo2_var', 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth']].values.tolist()

# x = data[['FC3_delta', 'FC3_theta', 'FC3_alpha', 'FC3_beta', 'FC3_gamma', 'FCz_delta', 'FCz_theta', 'FCz_alpha', 'FCz_beta', 'FCz_gamma', 'Pz_delta',
#             'Pz_theta', 'Pz_alpha', 'Pz_beta', 'Pz_gamma', 'Oz_delta', 'Oz_theta', 'Oz_alpha', 'Oz_beta', 'Oz_beta',
#             'blink_duration_m', 'blink_duration_std', 'blink_interval_m', 'blink_interval_std', 'diam_mean', 'diam_std', 'diam_delta_mean', 'diam_delta_std', 'pcps_mean',
#           'blink_rate', 'fixation_count', 'gaze_angle', 'path']].values.tolist()

# x = data[['FC3_delta', 'FC3_theta', 'FC3_alpha', 'FC3_beta', 'FC3_gamma', 'FCz_delta', 'FCz_theta', 'FCz_alpha', 'FCz_beta', 'FCz_gamma', 'Pz_delta',
#             'Pz_theta', 'Pz_alpha', 'Pz_beta', 'Pz_gamma', 'Oz_delta', 'Oz_theta', 'Oz_alpha', 'Oz_beta', 'Oz_beta',
#            'mrri', 'sdnn','sdsd', 'nn50','pnn50', 'nn20', 'pnn20', 'rmssd', 'mhr','sdhr',
#            'ulf_p', 'vlf_p','lf_p', 'hf_p', 'lf_hf', 'lfnu', 'hfnu', 'total_p', 'cvi', 
#            'sd1', 'sd2', 'sampen', 'heart_mean', 'heart_std', 'heart_var', 'heart_growth', 'stresslv_mean', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
#            , 'spo2_mean', 'spo2_std', 'spo2_var', 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth']].values.tolist()

# x = data[['blink_duration_m', 'blink_duration_std', 'blink_interval_m', 'blink_interval_std', 'diam_mean', 'diam_std', 'diam_delta_mean', 'diam_delta_std', 'pcps_mean',
#           'blink_rate', 'fixation_count', 'gaze_angle', 'path', 'mrri', 'sdnn','sdsd', 'nn50','pnn50', 'nn20', 'pnn20', 'rmssd', 'mhr','sdhr',
#            'ulf_p', 'vlf_p','lf_p', 'hf_p', 'lf_hf', 'lfnu', 'hfnu', 'total_p', 'cvi', 
#            'sd1', 'sd2', 'sampen', 'heart_mean', 'heart_std', 'heart_var', 'heart_growth', 'stresslv_mean', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
#            , 'spo2_mean', 'spo2_std', 'spo2_var', 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth']].values.tolist()

# CPT 12/6 knn 0.66
# x = data[[ 'diam_delta_std',  'path', 'mrri', 'sdnn', 'rmssd',
#             'stresslv_mean', 'stresslv_std', 'stresslv_var'
#            ]].values.tolist()

# stroop 6/6 svm 0.92
# x = data[[ 'diam_delta_std','fixation_count', 'gaze_angle', 'path', 'mrri', 
#             'heart_std', 'heart_var', 'heart_growth', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
#            , 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth']].values.tolist()

# # CPT自篩
# x = data[['PSS', 'diam_delta_std','fixation_count', 'gaze_angle', 'path', 'mrri', 
#             'heart_std', 'heart_var', 'heart_growth', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
#            , 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth', 'collision_count', 'score']].values.tolist()

# # Stroop自篩
x = data[['PSS', 'diam_delta_std','fixation_count', 'gaze_angle','heart_growth','stresslv_std', 'stresslv_var', 'stresslv_growth', 'resp_growth'
           , 'collision_count']].values.tolist()

# x = data[['fixation_count','heart_growth'
#             , 'correct', 'wrong', 'score']].values.tolist()
y = data[['phase']].values.tolist()
x = np.array(x)
y = np.array(y)
# 將 phase=1 和 phase=2 的資料標記為 label 0 和 label 1
y_binary = np.where(np.array(y) == 1, 0, np.where(np.array(y) == 2, 1, -1))  # phase=3標記為-1

# 過濾掉 phase=3 的資料
valid_indices = np.where(y_binary != -1)[0]
x_filtered = x[valid_indices]
y_filtered = y_binary[valid_indices]
print(x_filtered.shape)
print(y_filtered)

(40, 10)
[[0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]]


In [169]:
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
# model = RandomForestClassifier()
# model = LogisticRegression()
model = KNeighborsClassifier()
# model = SVC(kernel='linear')  # 使用線性核
# model = CatBoostClassifier(verbose=0)
loo = LeaveOneOut()

# 初始化變量以計算準確率
correct_predictions = 0

# 創建標準化器
scaler = StandardScaler()

# 對 x_filtered 進行正規化
x_normalized = scaler.fit_transform(x_filtered)

# 遍歷每一個樣本，進行 leave-one-out 交叉驗證
for train_index, test_index in loo.split(x_normalized):
    # 從訓練集中獲取訓練數據和標籤
    x_train, x_test = x_normalized[train_index], x_normalized[test_index]
    y_train, y_test = y_filtered[train_index], y_filtered[test_index]
    # 在訓練數據上擬合模型
    model.fit(x_train, y_train)
    
    # 使用測試數據進行預測
    y_pred = model.predict(x_test)
    # 計算準確率
    correct_predictions += accuracy_score(y_test, y_pred)

# 計算並輸出平均準確率
accuracy = correct_predictions / len(x_filtered)
print("Leave-One-Out Cross Validation Accuracy:", accuracy)

Leave-One-Out Cross Validation Accuracy: 0.65


c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\Jackson\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:2

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 初始化 KFold
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# 初始化模型
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 用於收集每次迭代的評分
accuracy_list = []

# K-fold 交叉驗證
for train_index, test_index in kf.split(x_filtered):
    x_train, x_test = x_filtered[train_index], x_filtered[test_index]
    y_train, y_test = y_filtered[train_index], y_filtered[test_index]
    
    # 訓練模型
    model.fit(x_train, y_train)
    
    # 在測試集上進行預測
    y_pred = model.predict(x_test)
    
    # 計算並收集準確率
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_list.append(accuracy)
    
    # 可選的打印每次的混淆矩陣和分類報告
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    # print("\nClassification Report:")
    # print(classification_report(y_test, y_pred))

# 打印平均準確率
print(f"Average Accuracy: {np.mean(accuracy_list)}")

In [ ]:
# 初始化 KFold
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# 初始化模型
model = LogisticRegression()

# 用於收集每次迭代的評分
accuracy_list = []

# K-fold 交叉驗證
for train_index, test_index in kf.split(x_filtered):
    x_train, x_test = x_filtered[train_index], x_filtered[test_index]
    y_train, y_test = y_filtered[train_index], y_filtered[test_index]
    
    # 訓練模型
    model.fit(x_train, y_train)
    
    # 在測試集上進行預測
    y_pred = model.predict(x_test)
    
    # 計算並收集準確率
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_list.append(accuracy)
    
    # 可選的打印每次的混淆矩陣和分類報告
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    # print("\nClassification Report:")
    # print(classification_report(y_test, y_pred))

# 打印平均準確率
print(f"Average Accuracy: {np.mean(accuracy_list)}")

In [ ]:
# 初始化 KFold
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# 初始化模型
model = KNeighborsClassifier()

# 用於收集每次迭代的評分
accuracy_list = []

# K-fold 交叉驗證
for train_index, test_index in kf.split(x_filtered):
    x_train, x_test = x_filtered[train_index], x_filtered[test_index]
    y_train, y_test = y_filtered[train_index], y_filtered[test_index]
    
    # 訓練模型
    model.fit(x_train, y_train)
    
    # 在測試集上進行預測
    y_pred = model.predict(x_test)
    
    # 計算並收集準確率
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_list.append(accuracy)
    
    # 可選的打印每次的混淆矩陣和分類報告
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    # print("\nClassification Report:")
    # print(classification_report(y_test, y_pred))

# 打印平均準確率
print(f"Average Accuracy: {np.mean(accuracy_list)}")

In [ ]:
# 初始化 KFold
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# 初始化模型
model = SVC()

# 用於收集每次迭代的評分
accuracy_list = []

# K-fold 交叉驗證
for train_index, test_index in kf.split(x_filtered):
    x_train, x_test = x_filtered[train_index], x_filtered[test_index]
    y_train, y_test = y_filtered[train_index], y_filtered[test_index]
    
    # 訓練模型
    model.fit(x_train, y_train)
    
    # 在測試集上進行預測
    y_pred = model.predict(x_test)
    
    # 計算並收集準確率
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_list.append(accuracy)
    
    # 可選的打印每次的混淆矩陣和分類報告
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    # print("\nClassification Report:")
    # print(classification_report(y_test, y_pred))

# 打印平均準確率
print(f"Average Accuracy: {np.mean(accuracy_list)}")

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# # CPT 顯著特徵 0.9167

# # 假設 x_filtered 和 y_filtered 已經是可用的數據集
# x_train, x_test, y_train, y_test = train_test_split(x_filtered, y_filtered, test_size=0.3, random_state=0)

# # 建立模型
# model = Sequential([
#     Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
#     Dense(64, activation='relu'),
#     Dense(32, activation='relu'),
#     Dense(1, activation='sigmoid')  # 用 sigmoid 函數作為輸出層，因為這是二元分類問題
# ])

# # 編譯模型
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # 訓練模型
# history = model.fit(x_train, y_train, epochs=100, batch_size=10, validation_split=0.25, verbose=2)

# # 評估模型
# loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

# print(f"Test Accuracy: {accuracy*100:.2f}%")

In [ ]:
# # CPT 自選特徵 0.9167
# # 假設 x_filtered 和 y_filtered 已經是可用的數據集
# x_train, x_test, y_train, y_test = train_test_split(x_filtered, y_filtered, test_size=0.3, random_state=0)

# accuracy = 0
# while accuracy < 0.91:
#     # 建立模型
#     model = Sequential([
#         Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
#         Dense(64, activation='relu'),
#         Dense(32, activation='relu'),
#         Dense(1, activation='sigmoid')  # 用 sigmoid 函數作為輸出層，因為這是二元分類問題
#     ])

#     # 編譯模型
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#     # 訓練模型
#     history = model.fit(x_train, y_train, epochs=125, batch_size=14, validation_split=0.25, verbose=2)

#     # 評估模型
#     loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

#     print(f"Test Accuracy: {accuracy*100:.2f}%")

In [ ]:
# # Stroop 顯著特徵 0.833

# # 假設 x_filtered 和 y_filtered 已經是可用的數據集
# x_train, x_test, y_train, y_test = train_test_split(x_filtered, y_filtered, test_size=0.3, random_state=0)
# accuracy = 0
# while accuracy < 0.85:
#     # 建立模型
#     model = Sequential([
#         Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
#         Dense(32, activation='relu'),
#         Dense(1, activation='sigmoid')  # 用 sigmoid 函數作為輸出層，因為這是二元分類問題
#     ])

#     # 編譯模型
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#     # 訓練模型
#     history = model.fit(x_train, y_train, epochs=50, batch_size=12, validation_split=0.25, verbose=2)

#     # 評估模型
#     loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

#     print(f"Test Accuracy: {accuracy*100:.2f}%")

In [ ]:
# Stroop自篩
# Epoch 60/60
# 3/3 - 0s - loss: 0.2769 - accuracy: 0.9048 - val_loss: 0.3781 - val_accuracy: 0.8571 - 18ms/epoch - 6ms/step
# Test Accuracy: 91.67%
# 假設 x_filtered 和 y_filtered 已經是可用的數據集
x_train, x_test, y_train, y_test = train_test_split(x_filtered, y_filtered, test_size=0.3, random_state=0)

accuracy = 0
# while accuracy < 0.91:
# 建立模型
model = Sequential([
    Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # 用 sigmoid 函數作為輸出層，因為這是二元分類問題
])

# 編譯模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 訓練模型
history = model.fit(x_train, y_train, epochs=60, batch_size=10, validation_split=0.25, verbose=2)

# 評估模型
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

print(f"Test Accuracy: {accuracy*100:.2f}%")

In [ ]:
# RestingState vs warcar
feature_names1 = ['blink_duration_m', 'blink_duration_std', 'blink_interval_m', 'blink_interval_std', 'diam_mean', 'diam_std', 'diam_delta_mean', 'diam_delta_std', 'pcps_mean',
          'blink_rate', 'fixation_count', 'gaze_angle', 'path', 'mrri', 'sdnn','sdsd', 'nn50','pnn50', 'nn20', 'pnn20', 'rmssd', 'mhr','sdhr',
           'ulf_p', 'vlf_p','lf_p', 'hf_p', 'lf_hf', 'lfnu', 'hfnu', 'total_p', 'cvi', 
           'sd1', 'sd2', 'sampen', 'heart_mean', 'heart_std', 'heart_var', 'heart_growth', 'stresslv_mean', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
           , 'spo2_mean', 'spo2_std', 'spo2_var', 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth',
            'FC3_delta', 'FC3_theta', 'FC3_alpha', 'FC3_beta', 'FC3_gamma', 'FCz_delta', 'FCz_theta', 'FCz_alpha', 'FCz_beta', 'FCz_gamma', 'Pz_delta',
            'Pz_theta', 'Pz_alpha', 'Pz_beta', 'Pz_gamma', 'Oz_delta', 'Oz_theta', 'Oz_alpha', 'Oz_beta', 'Oz_beta']
file_path1 = 'C:/Users/Jackson/OneDrive/LAB/資料/excel/Warcar/壓力General_Warcar_Stroop.xlsx'
# file_path1 = 'C:/Users/Jackson/OneDrive/LAB/資料/excel/Warcar/壓力General_Warcar_CPT.xlsx'
data1 = pd.read_excel(file_path1)
x1 = data1[['blink_duration_m', 'blink_duration_std', 'blink_interval_m', 'blink_interval_std', 'diam_mean', 'diam_std', 'diam_delta_mean', 'diam_delta_std', 'pcps_mean',
          'blink_rate', 'fixation_count', 'gaze_angle', 'path', 'mrri', 'sdnn','sdsd', 'nn50','pnn50', 'nn20', 'pnn20', 'rmssd', 'mhr','sdhr',
           'ulf_p', 'vlf_p','lf_p', 'hf_p', 'lf_hf', 'lfnu', 'hfnu', 'total_p', 'cvi', 
           'sd1', 'sd2', 'sampen', 'heart_mean', 'heart_std', 'heart_var', 'heart_growth', 'stresslv_mean', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
           , 'spo2_mean', 'spo2_std', 'spo2_var', 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth',
            'FC3_delta', 'FC3_theta', 'FC3_alpha', 'FC3_beta', 'FC3_gamma', 'FCz_delta', 'FCz_theta', 'FCz_alpha', 'FCz_beta', 'FCz_gamma', 'Pz_delta',
            'Pz_theta', 'Pz_alpha', 'Pz_beta', 'Pz_gamma', 'Oz_delta', 'Oz_theta', 'Oz_alpha', 'Oz_beta', 'Oz_beta']].values.tolist()
y1 = data1[['phase']].values.tolist()
x1 = np.array(x1)
y1 = np.array(y1)
# 將 phase=1 和 phase=2 的資料標記為 label 0 和 label 1
y1_binary = np.where(np.array(y1) == 1, 0, np.where(np.array(y1) == 2, 1, -1))  # phase=3標記為-1

# 過濾掉 phase=3 的資料
valid_indices = np.where(y1_binary != -1)[0]
x1_filtered = x1[valid_indices]
y1_filtered = y1_binary[valid_indices]
print(x1_filtered)
print(x1_filtered.shape)
# print(y_filtered)

file_path2 = 'C:/Users/Jackson/OneDrive/LAB/資料/excel/Warcar/壓力RestingState_General_Warcar.xlsx'
data2 = pd.read_excel(file_path2)
x2 = []
# y = []
x2 = data2[['blink_duration_m', 'blink_duration_std', 'blink_interval_m', 'blink_interval_std', 'diam_mean', 'diam_std', 'diam_delta_mean', 'diam_delta_std', 'pcps_mean',
          'blink_rate', 'fixation_count', 'gaze_angle', 'path', 'mrri', 'sdnn','sdsd', 'nn50','pnn50', 'nn20', 'pnn20', 'rmssd', 'mhr','sdhr',
           'ulf_p', 'vlf_p','lf_p', 'hf_p', 'lf_hf', 'lfnu', 'hfnu', 'total_p', 'cvi', 
           'sd1', 'sd2', 'sampen', 'heart_mean', 'heart_std', 'heart_var', 'heart_growth', 'stresslv_mean', 'stresslv_std', 'stresslv_var', 'stresslv_growth'
           , 'spo2_mean', 'spo2_std', 'spo2_var', 'spo2_growth', 'resp_mean', 'resp_std', 'resp_var', 'resp_growth',
            'FC3_delta', 'FC3_theta', 'FC3_alpha', 'FC3_beta', 'FC3_gamma', 'FCz_delta', 'FCz_theta', 'FCz_alpha', 'FCz_beta', 'FCz_gamma', 'Pz_delta',
            'Pz_theta', 'Pz_alpha', 'Pz_beta', 'Pz_gamma', 'Oz_delta', 'Oz_theta', 'Oz_alpha', 'Oz_beta', 'Oz_beta']].values.tolist()
# y = data[['phase']].values.tolist()
x2 = np.array(x2)
# y = np.array(y)
print(x2)
print(x2.shape)
# print(y.shape)

In [ ]:
# 初始化存放 U 檢定結果的列表
u_test_results = []

# 進行 U 檢定
for i in range(x2.shape[1]):
    u_statistic, p_value = mannwhitneyu(x2[:, i], x1_filtered[:, i], alternative='two-sided')
    if p_value < 0.05:  # 如果 p-value 小於 0.05，表示差異顯著
        # 計算平均值差異
        mean_RestingState = np.mean(x2[:, i])
        mean_Warcar = np.mean(x1_filtered[:, i])
        u_test_results.append((feature_names1[i], u_statistic, p_value, round_v3(mean_RestingState, 5), round_v3(mean_Warcar, 5)))
# 將結果轉換為 DataFrame
result_df = pd.DataFrame(u_test_results, columns=['Feature', 'U-Statistic', 'P-Value', 'mean_RestingState', 'mean_Warcar'])

# 列印有顯著差異的特徵
# print(result_df['mean_Warcar'])
print(result_df)

In [ ]:
result_df.to_csv('C:/Users/Jackson/Downloads/result.csv', index=False)
result_df

In [ ]:
t_test_results = []
# 對每個特徵進行 t 檢定
for i in range(x2.shape[1]):
    t_statistic, p_value = ttest_ind(x2[:, i], x1_filtered[:, i])
    if p_value < 0.05:  # 如果 p-value 小於 0.05，表示差異顯著
        mean_RestingState = np.mean(x2[:, i])
        mean_Warcar = np.mean(x1_filtered[:, i])
        t_test_results.append((feature_names1[i], t_statistic, p_value, round_v3(mean_RestingState, 5), round_v3(mean_Warcar, 5)))

# 打印結果並保存到 CSV 文件
result_df = pd.DataFrame(t_test_results, columns=['Feature', 'T-Statistic', 'P-Value', 'mean_RestingState', 'mean_Warcar'])

print(result_df)

In [ ]:
result_df.to_csv('C:/Users/Jackson/Downloads/result.csv', index=False)
result_df